# Fish Speech Fine Tune

This is just a convenience thing for following Fish's finetune in colab. I didn't have enough VRAM locally so I use this notebook for cloud training.

## For Linux

In [ ]:
import locale

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

## 1. Prepare Env and Data

TODO - use fish audio processing, right now just uploading pre-prepared, see: https://speech.fish.audio/finetune/

<hr>

In [ ]:
!git clone https://github.com/fishaudio/fish-speech

Cloning into 'fish-speech'...
remote: Enumerating objects: 4824, done.
remote: Counting objects: 100% (1399/1399), done.
remote: Compressing objects: 100% (328/328), done.
remote: Total 4824 (delta 1164), reused 1153 (delta 1057), pack-reused 3425 (from 1)
Receiving objects: 100% (4824/4824), 17.96 MiB | 1.97 MiB/s, done.
Resolving deltas: 100% (3166/3166), done.


In [ ]:
# Install torch, etc. etc.
!pip install torch torchvision torchaudio

In [ ]:
!apt install libsox-dev ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-all
  libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse
  libsox3 libwavpack1
Suggested packages:
  libaudio2 libsndio6.1
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-dev
  libsox-fmt-all libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss
  libsox-fmt-pulse libsox3 libwavpack1
0 upgraded, 16 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,053 kB of archives.
After this operation, 4,061 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao-common all 1.2.2+20180113-1.1ub

In [ ]:
# Install fish-speech reqs
!pip install -e /content/fish-speech/.

Obtaining file:///content/fish-speech
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.1/298.1 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of vector-quantize-pytorch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of vector-quantiz

In [ ]:
import os

os.makedirs('/content/fish-speech/data/SPK1', exist_ok=True)
os.chdir("/content/fish-speech/")

You then drag your archive/files with both the .mp3 and .lab files to the folder you created

In [ ]:
!cd /content/fish-speech/data/SPK1 && tar -xvf archive.tar > /dev/null 2>&1

## 2. Batch extraction of semantic tokens

<hr>


Make sure you have downloaded the VQGAN weights. If not, run the following command:



In [ ]:
!huggingface-cli download fishaudio/fish-speech-1.4 --local-dir checkpoints/fish-speech-1.4

Fetching 8 files:   0% 0/8 [00:00<?, ?it/s]Downloading 'special_tokens_map.json' to 'checkpoints/fish-speech-1.4/.cache/huggingface/download/special_tokens_map.json.afaceb358180a0ba4d0ae9bd3b915b1ed43aa99d.incomplete'

config.json: 100% 493/493 [00:00<00:00, 1.99MB/s]

README.md: 100% 1.39k/1.39k [00:00<00:00, 14.7MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/config.json
Download complete. Moving file to checkpoints/fish-speech-1.4/README.md

special_tokens_map.json: 100% 449/449 [00:00<00:00, 3.47MB/s]

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 13.7MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/special_tokens_map.json
Download complete. Moving file to checkpoints/fish-speech-1.4/.gitattributes
Fetching 8 files:  12% 1/8 [00:00<00:01,  5.56it/s]
tokenizer.json:   0% 0.00/1.60M [00:00<?, ?B/s]

tokenizer_config.json: 100% 1.85k/1.85k [00:00<00:00, 17.1MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/tokenizer_config.json

You can then run the following command to extract semantic tokens:

In [ ]:
!python tools/vqgan/extract_vq.py data \
    --num-workers 1 --batch-size 16 \
    --config-name "firefly_gan_vq" \
    --checkpoint-path "checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

2024-10-17 02:02:32.149 | INFO     | __main__:main:184 | RANK: 0 / 1 - Starting worker
Found 301 files
2024-10-17 02:02:32.165 | INFO     | __main__:main:195 | RANK: 0 / 1 - Processing 301/301 files
/usr/local/lib/python3.10/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:457: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.10/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:642: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.10/dist-packages/vector_quantize_pytorch/finite_scalar_quantization.py:162: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/content/fish-speech/tools/vqgan/extract_vq.py:52: Fut

## 3. Pack the dataset into protobuf

<hr>

In [ ]:
!python tools/llama/build_dataset.py \
    --input "data" \
    --output "data/protos" \
    --text-extension .lab \
    --num-workers 16

0it [00:00, ?it/s]
Loading data: 301it [00:00, 82343.17it/s]

Grouping data: 100% 301/301 [00:00<00:00, 31281.39it/s]
2024-10-17 02:03:04.522 | INFO     | __main__:task_generator_folder:46 - Found 1 groups in data, ['data/SPK1']...
1it [00:00,  4.45it/s]
2024-10-17 02:03:04.740 | INFO     | __main__:main:165 - Finished writing 1 shards to data/protos


After the command finishes executing, you should see the quantized-dataset-ft.protos file in the data directory.

## 4. Fine-tuning with LoRA


Similarly, make sure you have downloaded the LLAMA weights. If not, run the following command:

In [ ]:
!huggingface-cli download fishaudio/fish-speech-1.4 --local-dir checkpoints/fish-speech-1.4

Fetching 8 files: 100% 8/8 [00:00<00:00, 6319.10it/s]
/content/fish-speech/checkpoints/fish-speech-1.4


Finally, you can start the fine-tuning by running the following command:



> Note: You can modify the training parameters such as batch_size, gradient_accumulation_steps, etc. to fit your GPU memory by modifying fish_speech/configs/text2semantic_finetune.yaml.


In [ ]:
!python fish_speech/train.py --config-name text2semantic_finetune \
    project=$project \
    +lora@model.model.lora_config=r_8_alpha_16

[2024-10-17 02:03:27,931][__main__][INFO] - [rank: 0] Instantiating datamodule <fish_speech.datasets.semantic.SemanticDataModule>
[2024-10-17 02:03:28,104][numexpr.utils][INFO] - NumExpr defaulting to 12 threads.
[2024-10-17 02:03:28,252][datasets][INFO] - PyTorch version 2.4.1+cu121 available.
[2024-10-17 02:03:28,253][datasets][INFO] - TensorFlow version 2.17.0 available.
[2024-10-17 02:03:28,254][datasets][INFO] - JAX version 0.4.33 available.
[2024-10-17 02:03:28,694][__main__][INFO] - [rank: 0] Instantiating model <fish_speech.models.text2semantic.lit_module.TextToSemantic>
[2024-10-17 02:03:29,054][fish_speech.models.text2semantic.llama][INFO] - [rank: 0] Override max_seq_len to 4096
[2024-10-17 02:03:29,097][fish_speech.models.text2semantic.llama][INFO] - [rank: 0] Loading model from checkpoints/fish-speech-1.4, config: DualARModelArgs(model_type='dual_ar', vocab_size=32000, n_layer=24, n_head=16, dim=1024, intermediate_size=4096, n_local_heads=2, head_dim=64, rope_base=1000000.

After training, you need to convert the LoRA weights to regular weights before performing inference.

In [ ]:
!ls results/checkpoints

step_000000600.ckpt  step_000000800.ckpt  step_000001000.ckpt
step_000000700.ckpt  step_000000900.ckpt


Make sure to match with one of these files, we suggest using the earliest checkpoint that meets your requirements, as they often perform better on out-of-distribution (OOD) data.

In [ ]:
!python tools/llama/merge_lora.py \
    --lora-config r_8_alpha_16 \
    --base-weight checkpoints/fish-speech-1.4 \
    --lora-weight results/checkpoints/step_000000600.ckpt \
    --output checkpoints/fish-speech-1.4-yth-lora/

2024-10-17 03:24:31.585 | INFO     | __main__:merge:23 - Merging checkpoints/fish-speech-1.4 and results/checkpoints/step_000000600.ckpt into checkpoints/fish-speech-1.4-yth-lora with r_8_alpha_16
2024-10-17 03:24:31.677 | INFO     | __main__:merge:31 - Loaded lora model with config LoraConfig(r=8, lora_alpha=16, lora_dropout=0.01)
/content/fish-speech/fish_speech/models/text2semantic/llama.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly a

In [ ]:
!ls results/checkpoints

step_000000600.ckpt  step_000000800.ckpt  step_000001000.ckpt
step_000000700.ckpt  step_000000900.ckpt


In [ ]:
!tar -czvf fish-speech-1.4-yth.tar.gz checkpoints/fish-speech-1.4-yth-lora

checkpoints/fish-speech-1.4-yth-lora/
checkpoints/fish-speech-1.4-yth-lora/model.pth
checkpoints/fish-speech-1.4-yth-lora/tokenizer.json
checkpoints/fish-speech-1.4-yth-lora/config.json
checkpoints/fish-speech-1.4-yth-lora/tokenizer_config.json
checkpoints/fish-speech-1.4-yth-lora/special_tokens_map.json
